In [ ]:
# ================================================
# SPECKLE REDUCTION FOR SAR IMAGES USING AUTOENCODER
# ================================================
# This script filters SAR satellite images using a pre-trained autoencoder model.
# Due to memory constraints, large images are processed in 512x512 patches.
# The model must be trained separately and saved as 'model.h5'.
# Dependencies: TensorFlow 2.15.0, Keras 2.15.0, OpenCV, Rasterio, NumPy
# ================================================

# ---- INSTALL REQUIRED DEPENDENCIES ----
!pip install --upgrade --force-reinstall tensorflow==2.15.0
!pip install --upgrade --force-reinstall keras==2.15.0
!pip install rasterio


In [ ]:
# ---- IMPORT LIBRARIES ----
import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from keras import backend
from glob import glob
import rasterio
from google.colab import drive

# ---- MOUNT GOOGLE DRIVE ----
# Required for loading the model and accessing SAR images
drive.mount('/content/drive')


In [ ]:
# ---- PREPROCESSING FUNCTION ----
def preprocess(array):
    """
    Normalizes and reshapes an array to the format expected by the autoencoder.

    Args:
        array (numpy.ndarray): Array of shape (N, 512, 512)

    Returns:
        numpy.ndarray: Reshaped array of shape (N, 512, 512, 1), dtype float32
    """
    array = array.astype("float32")
    array = np.reshape(array, (len(array), 512, 512, 1))
    return array

# ---- LOAD PRETRAINED AUTOENCODER MODEL ----
autoencoder_loaded = keras.models.load_model('/content/drive/MyDrive/model.h5')

# ---- DEFINE PROCESSING ZONES ----
# Each zone is defined by bounding box coordinates (not directly used here)
zones = {
    "CONFLUENCIA": {
        "lat1": 8.037527,
        "lon1": -74.851446,
        "lat2": 8.113206,
        "lon2": -74.746023
    },
}

In [ ]:

# ---- MAIN PROCESSING LOOP ----
for zone in zones.keys():
    # Get list of all .tif images in the VV_VISUAL folder
    visuals = glob(f'/content/drive/MyDrive/SAR_BAJO_CAUCA/{zone}/VV_VISUAL/*.tif')
    visuals.sort()  # Sort by filename (date)

    for visual in visuals:
        # Read the SAR image using Rasterio
        with rasterio.open(visual) as src:
            data = src.read()                  # Read all bands
            image0_profile = src.profile       # Georeferencing information
            image0_meta = src.meta             # General metadata

        # Extract and normalize the first band
        img = data[0, :, :]                    # Use the first band only
        img = img.astype(np.single) / 255.0    # Normalize to range [0, 1]

        # Create empty output image to store filtered results
        new = np.zeros(img.shape, dtype=np.uint8)
        h, w = new.shape
        patch_size = 512

        # ---- PROCESS IMAGE IN 512x512 PATCHES ----
        for i in range(0, h, patch_size):
            if i > (h - patch_size):
                i = h - patch_size  # Prevent going out of bounds
            for j in range(0, w, patch_size):
                if j > (w - patch_size):
                    j = w - patch_size

                # Extract 512x512 patch from original image
                roi = img[i:i+patch_size, j:j+patch_size]
                val1 = cv2.resize(roi, (patch_size, patch_size))  # Ensure exact shape

                # Prepare the patch for prediction
                sfs = [val1]
                valid = np.array(sfs, dtype=np.single)
                valida = preprocess(valid)

                # ---- APPLY AUTOENCODER FILTER ----
                prediction = autoencoder_loaded.predict(valida)
                imgfilt = prediction.reshape(patch_size, patch_size) * 255
                imgfilt = imgfilt.astype(np.uint8)

                # Insert filtered patch into the final image
                new[i:i+patch_size, j:j+patch_size] = imgfilt

        # ---- SAVE FILTERED IMAGE ----
        filename = visual.split('/')[-1]  # Extract the filename
        new_profile = image0_profile.copy()
        new_profile.update(count=1, dtype=rasterio.uint8)  # Update for single-band uint8 image

        # Output file path
        output_path = f'/content/drive/MyDrive/{zone}/VV_VISUAL_FILT/' + filename

        # Write the filtered image using Rasterio
        with rasterio.open(output_path, 'w', **new_profile) as dst:
            dst.write(new, 1)  # Write to the first band

        print(f'{zone} {filename} successfully filtered and saved!')